In [ ]:
import os
import sys
import matplotlib.pyplot as plt
import numpy as np
import matplotlib.pyplot as plt
from pathlib import Path
from dotenv import load_dotenv
from PIL import Image

root = Path(os.getcwd()).parents[1].resolve()

sys.path.insert(0, str(root))

assert load_dotenv(".env")

In [ ]:
from datetime import datetime

datetime.utcnow().strftime(os.environ['DATETIME_FMT'])

# Glance

In [ ]:
data = root.joinpath("data/affectnet")
group = data.joinpath("test")

image_dir = group.joinpath("images")
label_dir = group.joinpath("annotations")

image_path_list = list(image_dir.glob("*.jpg"))
label_path_list = list(label_dir.glob("*.npy"))

In [ ]:
sample = image_path_list[0]
sample

In [ ]:
sample_nor = np.array(Image.open(sample)).astype(np.float32) / 255.0

plt.imshow(sample_nor)
plt.show()

# Extact Label

## select annotation files containing emotion labels

In [ ]:
emotion_label_file_paths = [path for path in label_path_list if path.name.split("_")[1].split(".")[0] == "exp"]
emotion_label_file_paths[:6]

## select labels and file names from annotation files

In [ ]:
emotion_label_file_names = [path.name.split("_")[0] for path in emotion_label_file_paths]
emotion_label_file_names[:6]

In [ ]:
emotion_labels = [int(np.load(file).tolist()[0]) for file in emotion_label_file_paths]
emotion_labels[:6]

# create corresponding directory for storing classified face images

In [ ]:
from core.utils.config import emotion

face_dir = group.joinpath("faces")
face_dir.mkdir(exist_ok=True)

for c in emotion.keys():
    face_dir.joinpath(c).mkdir(exist_ok=True) 

In [ ]:
affectnet_mapping = {
    0: "neutral", 1: "happy", 2: "sad", 3: "surprise", 4: "fear", 5: "disgust", 6: "angry"
}

In [ ]:
from shutil import move

for image_name, label in zip(emotion_label_file_names, emotion_labels):
    if label > 6:
        continue
    name = image_name + ".jpg"
    src = image_dir.joinpath(name) # get original file path

    if not src.exists():
        continue
    true_label = affectnet_mapping[label] # get category str

    dst = face_dir.joinpath(true_label, name) # create new file path

    move(src, dst)